In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import scvelo as scv
from scipy import stats
import os
import matplotlib
import pickle
%matplotlib inline
import palantir
import harmony
import random
np.random.seed(0)
random.seed(0)

In [3]:
import sys
sys.path.append('../Integrated_analysis/')
from Integrate import concat_for_harmony, adata_to_df

In [6]:
A1_2 = pd.read_csv('A1_2_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
A1_3 = pd.read_csv('A1_3_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
A1_4 = pd.read_csv('A1_4_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
A2_1 = pd.read_csv('A2_1_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
A2_2 = pd.read_csv('A2_2_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]


B1_1 = pd.read_csv('B2_1_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
B1_2 = pd.read_csv('B1_2_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
B2_1 = pd.read_csv('B2_1_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]
B2_2 = pd.read_csv('B2_2_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]

D2_1 = pd.read_csv('D2_1_measurements.csv', sep = '\t', skiprows=1, index_col = 0).iloc[:,:-1]

In [9]:
tp = [*['8/10']*len(A1_2.columns),
*['8/10']*len(A1_3.columns),
*['8/10']*len(A1_4.columns),
*['18/20']*len(A2_1.columns),
*['18/20']*len(A2_2.columns),
*['12']*len(B1_1.columns),
*['12']*len(B1_2.columns),
*['13']*len(B2_1.columns),
*['13']*len(B2_2.columns),
*['27']*len(D2_1.columns)]

ph = [*['A1_2']*len(A1_2.columns),
*['A1_3']*len(A1_3.columns),
*['A1_4']*len(A1_4.columns),
*['A2_1']*len(A2_1.columns),
*['A2_2']*len(A2_2.columns),
*['B1_1']*len(B1_1.columns),
*['B1_2']*len(B1_2.columns),
*['B2_1']*len(B2_1.columns),
*['B2_2']*len(B2_2.columns),
*['D2_1']*len(D2_1.columns)]

In [10]:
merge = pd.concat([A1_2, A1_3, A1_4, A2_1,A2_2, B1_1, B1_2, B2_1, B2_2, D2_1], axis = 1).replace(np.NaN, 0)

In [11]:
merge = sc.AnnData(merge.T)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [12]:
merge.obs['Dev_time'] = tp
merge.obs['Photo'] = ph


In [13]:
merge

AnnData object with n_obs × n_vars = 12732 × 31
    obs: 'Dev_time', 'Photo'

In [14]:
def preprocess(adata, N_counts = 8):
    #adata = sc.AnnData(df.T)
    sc.pp.filter_cells(adata, min_genes=1)
    sc.pp.filter_genes(adata, min_cells=3)
    sc.pp.calculate_qc_metrics(adata,  percent_top=None, log1p=False, inplace=True)
    adata = adata[adata.obs.total_counts > N_counts, :] #B1 = 5, B2_1 = 10
    sc.pp.normalize_total(adata, target_sum=1e4)
    adata.raw = adata
    #sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata, n_neighbors=40, n_pcs=3)
    sc.tl.umap(adata)
    return adata

In [15]:
merge_processed = preprocess(merge)

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/opt/conda/lib/python3.7/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
Observation names are not unique. To m

In [16]:
sns.boxplot(x = 'Dev_time', y = 'total_counts', data = merge_processed.obs, fliersize = 0)
plt.ylim(0, 200)

/opt/conda/lib/python3.7/site-packages/seaborn/categorical.py:402: UserWarning: Use the colorbar set_ticks() method instead.
  ax.set_xticks(np.arange(len(self.plot_data)))


(0.0, 200.0)

In [17]:
sc.tl.leiden(merge_processed, resolution = 0.4)

In [18]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
sc.pl.umap(merge_processed, color=['leiden'], ax = ax)

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
... storing 'Dev_time' as categorical
... storing 'Photo' as categorical


In [81]:
test = merge_processed.to_df()

In [82]:
test['Dev_time'] = merge_processed.obs.Dev_time

In [89]:
test2 = test.groupby('Dev_time').mean()

In [ ]:
genes = ['sox3_S',
         'ctcf_S' ,
         'has1_S',         
         'spdef_S','tp63_L', 'pfn1_L', 
         'mcidas_L','foxj1_L', 'tekt2_S',
         'grhl3_L','itln1_L' 
        ]

test3 = test2[genes]

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
scaler2 = MinMaxScaler()
scaled = pd.DataFrame(data = scaler.fit_transform(test3), index = test3.index, columns = test3.columns).T
scaled2 = pd.DataFrame(data = scaler2.fit_transform(test3), index = test3.index, columns = test3.columns).T


#plt.figure(figsize=(3,5))
fig, ax = plt.subplots(figsize=(4,5))
#res = 
sns.heatmap(scaled, cmap = 'RdBu_r', ax = ax)
# make frame visible
for _, spine in ax.spines.items():
    spine.set_visible(True)
    
ax.hlines(np.array([2, 3, 6, 9]) * 1, *ax.get_xlim(), linestyles = 'dashed')
plt.tight_layout()
plt.savefig('heatmap_standard.pdf')


fig, ax = plt.subplots(figsize=(4,5))
#res = 
sns.heatmap(scaled2, cmap = 'Blues', ax = ax)
# make frame visible
for _, spine in ax.spines.items():
    spine.set_visible(True)
    
ax.hlines(np.array([2, 3, 6, 9]) * 1, *ax.get_xlim(), linestyles = 'dashed')
plt.tight_layout()
plt.savefig('heatmap_minmax.pdf')